In [ ]:
import pandas as pd
import json
import re
import requests
from bs4 import BeautifulSoup

def getSoupOfDrugRequest(drug, page):
    #headers = {'content-type': 'text/html'}
    url = 'http://base-donnees-publique.medicaments.gouv.fr/index.php'
    data = {
    'choixRecherche':"medicament",
    'page':int(page),
    'affliste':1, 
    'affNumero':0,
    'isAlphabet':0,
    'inClauseSubst':0,
    'nomSubstances':"",
    'btnMedic.x': '12',
    'btnMedic.y': '5',
    'btnMedic': 'Rechercher',
    'typeRecherche':0,
    'txtCaracteres':drug,
    'radLibelle':2,
    'txtCaracteresSub':"",
    'radLibelleSub':4
    }
    response = requests.post(url, data) # les headers fonctionnent
    return BeautifulSoup(response.text,'html.parser')

In [ ]:
def getLiensFromSoup(soup):
    return soup.findAll(attrs={'class':'standart'})

# requete doliprane
soup = getSoupOfDrugRequest("doliprane", 1)

# nb pages
pageCountRaw = soup.findAll('div',{'class':'navBarGauche'})[0].text
pageCount    = re.compile(r'(\d+)$').search(pageCountRaw).group()

# recuperation liens medocs
links = list()
[links.extend(getLiensFromSoup(soup)) for soup in [getSoupOfDrugRequest("doliprane", page) for page in xrange(1,int(pageCount)+1)]]

# uniquement liens correspondant aux medocs, le tout dans un dataframe
drugsRawData =  pd.DataFrame([link.text for link in links if link['href'].startswith('extrait.php?specid')], columns=[u"rawData"])

# 3 cols de data (Médicament/Poids/Forme Galenique)
drugsDataInTreatement = pd.DataFrame(drugsRawData["rawData"].str.findall(r"^([A-Z\s]+)(.*),(.+)[\t+]$").tolist())
drugsData = pd.DataFrame(drugsDataInTreatement[0].tolist(), 
                         columns=[u'Drug name',u'Weight',u'Dosage form'])